In [1]:
import logging
import sys
import os
import pandas as pd
from pandas import Timestamp
# pylint: disable=import-error disable=wrong-import-position
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

print(os.path.dirname(os.path.abspath("__file__")))
import pydit
logger = pydit.setup_logging_info()
logger.info("Started")
fm= pydit.FileManager.getInstance()
fm.input_path="./demo_data/"
fm.output_path="./demo_data/"
fm.temp_path="./demo_data/"



c:\Users\User\git\pydit\examples


2022-05-12 06:21 INFO - Started


In [18]:
df = pd.DataFrame([
    [1, 'INV-220001', Timestamp('2022-01-01 00:00:00'), 'OPEN', 35.94,''],
    [2, 'INV-220002', Timestamp('2022-01-02 00:00:00'), 'OPEN', 99.99,'-5'],
    [3, 'INV-220003', Timestamp('2022-01-03 00:00:00'), 'CANCELLED', 13.0,'reinbursed 10.5'],
    [4, 'INV-220003', Timestamp('2022-01-04 00:00:00'), 'OPEN', float('nan'),''],
    [5, 'INV-220005', Timestamp('2022-01-04 00:00:00'), 'OPEN', 204.2,''],
    [6, 'INV-220006', Timestamp('2022-01-15 00:00:00'), 'OPEN', -4.2,'discount'],
    [7, float('nan'), Timestamp('2022-01-06 00:00:00'), float('nan'), 0.0,'Unknown error'],
    [8, 'INV-220007', Timestamp('2022-01-15 00:00:00'), 'PENDING', 50.4,''],
    [9, '', pd.NaT, 'ERROR', 0.0,'Timeout Error'],
    [10, 'INV-220007', Timestamp('2022-01-15 00:00:00'), 'PENDING', 50.4,'']], 
    columns=['id', 'ref', 'date_trans', 'status', 'amount','notes'])
df

,id,ref,date_trans,status,amount,notes
0,1,INV-220001,2022-01-01,OPEN,35.94,
1,2,INV-220002,2022-01-02,OPEN,99.99,-5
2,3,INV-220003,2022-01-03,CANCELLED,13.00,reinbursed 10.5
3,4,INV-220003,2022-01-04,OPEN,NaN,
4,5,INV-220005,2022-01-04,OPEN,204.20,
5,6,INV-220006,2022-01-15,OPEN,-4.20,discount
6,7,NaN,2022-01-06,NaN,0.00,Unknown error
7,8,INV-220007,2022-01-15,PENDING,50.40,
8,9,,NaT,ERROR,0.00,Timeout Error
9,10,INV-220007,2022-01-15,PENDING,50.40,


In [9]:
pydit.profile_dataframe(df)

,column,dtype,records,count_unique,nans,value_counts,max,min,sum,sum_abs,std,zeroes,empty_strings,cardinality_perc
0,id,int64,10,10,0,[],10,1,55.00,55.00,3.027650,0.0,NaN,1.0
1,ref,object,10,7,1,[],220007,220001,NaN,NaN,NaN,NaN,1.0,0.7
2,date_trans,datetime64[ns],10,6,1,[],2022-01-15 00:00:00,2022-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,0.6
3,status,object,10,4,1,"{'OPEN': 5, 'PENDING': 2, 'CANCELLED': 1, 'ERR...",NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.4
4,amount,float64,10,7,1,[],204.2,-4.2,449.73,458.13,66.877944,2.0,NaN,0.7
5,notes,object,10,5,0,[],10.5,5,NaN,NaN,NaN,NaN,6.0,0.5


In [10]:

pydit.check_blanks(df)

2022-05-05 13:20 INFO - Checking for blanks in id,ref,date_trans,status,amount,notes
2022-05-05 13:21 INFO - Total blanks found in each column:
{'id': 0, 'ref': 2, 'date_trans': 1, 'status': 1, 'amount': 3, 'notes': 6}


,id,ref,date_trans,status,amount,notes,id_blanks,ref_blanks,date_trans_blanks,status_blanks,amount_blanks,notes_blanks,has_blanks
0,1,INV-220001,2022-01-01,OPEN,35.94,,False,False,False,False,False,True,True
1,2,INV-220002,2022-01-02,OPEN,99.99,-5,False,False,False,False,False,False,False
2,3,INV-220003,2022-01-03,CANCELLED,13.00,reinbursed 10.5,False,False,False,False,False,False,False
3,4,INV-220003,2022-01-04,OPEN,NaN,,False,False,False,False,True,True,True
4,5,INV-220005,2022-01-04,OPEN,204.20,,False,False,False,False,False,True,True
5,6,INV-220006,2022-01-15,OPEN,-4.20,discount,False,False,False,False,False,False,False
6,7,NaN,2022-01-06,NaN,0.00,,False,True,False,True,True,True,True
7,8,INV-220007,2022-01-15,PENDING,50.40,,False,False,False,False,False,True,True
8,9,,NaT,ERROR,0.00,Timeout Error,False,True,True,False,True,False,True
9,10,INV-220007,2022-01-15,PENDING,50.40,,False,False,False,False,False,True,True


In [19]:
"""Function for performing coalesce."""
import logging
from typing import Optional, Union
import pandas as pd
logger = logging.getLogger(__name__)


def coalesce_columns(
    df: pd.DataFrame,
    *column_names,
    target_column_name: Optional[str] = None,
    default_value: Optional[Union[int, float, str]] = None,
    operation=None
) -> pd.DataFrame:
    if not column_names:
        return df
    print(column_names)
    if len(column_names) < 2:
        if isinstance(column_names[0],list) and len(column_names[0])>1:
            column_names=column_names[0]
        else:
            raise ValueError("The number of columns to coalesce should be a minimum of 2.")

    if isinstance(column_names, list) or isinstance(column_names, tuple):
        wrong_columns = [x for x in column_names if x not in df.columns]
        if wrong_columns:
            raise ValueError("Columns not in the dataframe:" + " ".join(wrong_columns))

    else:
        raise TypeError("Please provide a list of columns")

    if target_column_name:
        check_types("target_column_name", target_column_name, [str])

    if default_value:
        check_types("default_value", default_value, [int, float, str])

    if target_column_name is None:
        target_column_name = column_names[0]
    print("Target column name:", target_column_name)
    # bfill/ffill combo is faster than combine_first
    if operation is None:
        outcome = (
            df.filter(column_names).bfill(axis="columns").ffill(axis="columns").iloc[:, 0]
        )
    elif operation="concatenate":
        
    if outcome.hasnans and (default_value is not None):
        outcome = outcome.fillna(default_value)
    return df.assign(**{target_column_name: outcome})

coalesce_columns(df,["status","notes"])

(['status', 'notes'],)
Target column name: status


,id,ref,date_trans,status,amount,notes
0,1,INV-220001,2022-01-01,OPEN,35.94,
1,2,INV-220002,2022-01-02,OPEN,99.99,-5
2,3,INV-220003,2022-01-03,CANCELLED,13.00,reinbursed 10.5
3,4,INV-220003,2022-01-04,OPEN,NaN,
4,5,INV-220005,2022-01-04,OPEN,204.20,
5,6,INV-220006,2022-01-15,OPEN,-4.20,discount
6,7,NaN,2022-01-06,Unknown error,0.00,Unknown error
7,8,INV-220007,2022-01-15,PENDING,50.40,
8,9,,NaT,ERROR,0.00,Timeout Error
9,10,INV-220007,2022-01-15,PENDING,50.40,
